# <center><font color=navy> 7573 UNIX For Bioinformatics Final Project </font></center>

## <font color= green>Query One Entrez (5pts):</font>
1. Perform an analysis with both ‘ALK’ and  ‘EGFR’ as the search term AND human[organism].
2. Query gene.
3. Link to protein as the target, then parse the output to a file that has a delimiter which separates the columns based on Id, Source, Title, NlmUniqueID.
4. Obtain a summary with esummary.
5. Parse the xml file.  The final output file with have the Caption and SourceDb data as the content on each line of the file.

## <font color= green>Query Two Entrez (5pts):</font>
1. Query the pubmed db with  ‘ALK’ and  ‘EGFR’  as the search term .
2. Pipe the esearch query output to efetch and format it as `medline`.
3. Next step is to get the output to a file.

## <font color= green>Query Three Entrez (5pts):</font>
1. Perform a query of the clinvar db with  ‘ALK’and  ‘EGFR’ as the search term.
2. Use efetch to get the first 100 datapoints and format it in the docsum for file output.
3. Extract the xml file based on the DocumentSummary pattern. Use the "|" as the column separator and parse the elements Id accession title chr trait_name gene_sort band clinical_significance to a final text file.

## <font color= green>Question Four (5pts):</font>
Find the mutation/s that determines whether  ‘ALK’ and  ‘EGFR’  will be toxic or non-toxic for non-small cell squamous lung cancer(NSCLC) patient drug treatment. Create a table that has the headers  Toxic and Non-toxic. The lines will have the gene and the mutation.  Example, 
<font color=purple>ALK - 92C>G (p.Pro31Arg)</font>. The dash is the tab separator.

## <font color= green>Summary (10pts)</font>

Write up, using jupyter notebook markup language, a comparison and contrast of the two resources. You should address the following:
1. What are the strengths of Entrez EDirect?
2. How is it useful in analyzing a personal treatment for a given cancer?
3. What did you learn from the literature searches concerning the mutation for non-small cell squamous lung cancer(NSCLC)?
4. Which NCBI databases provided the information concerning the mutation and it’s sequence info? Describe the databases and the data that they hold.
5. How does the mutation inform the patients response to treatment?

## <font color=red>Before submitting the jupyter notebook, do the following:</font>
1. `Go to the 'Kernel' button and select 'Restart and Clear Output'.`
2. `Add all output files and the jupyter notebook to a folder.`
3. `Compress it then upload to Brightspace.`



#### My Solutions are below

#### **Question 1**
1. Perform an analysis with both ‘ALK’ and ‘EGFR’ as the search term AND human[organism].
2. Query gene.
3. Link to protein as the target, then parse the output to a file that has a delimiter which separates the columns based on Id, Source, Title, NlmUniqueID.
4. Obtain a summary with esummary.
5. Parse the xml file. The final output file with have the Caption and SourceDb data as the content on each line of the file.

***Note: I used the search `(ALK AND EGFR) AND human[organizm]` after the professor suggested it over an email***

In [ ]:
#answers question 3
!esearch -db gene -query "(ALK AND EGFR) AND human[organism]" | \
elink -target protein | \
efetch -format docsum | \
xtract -pattern DocumentSummary -element Id Source Title NlmUniqueID > output_1_1.txt

In [ ]:
#answers question 5
!esearch -db gene -query "(ALK AND EGFR)AND human[organism]"|\
elink -target protein|\
esummary|\
xtract -pattern DocumentSummary -element Caption SourceDb > output_1_2.txt

#### **Question 2**
1. Query the pubmed db with ‘ALK’ and ‘EGFR’ as the search term .
2. Pipe the esearch query output to efetch and format it as medline.
3. Next step is to get the output to a file.

**I needed to limit my results to the first 500 as NCBI was giving me an ERROR for too many results to write out to a document**

In [ ]:
!esearch -db pubmed -query "ALK AND EGFR"  |\
efetch -start 0 -stop 500 -format medline > output_2.txt

#### **Question 3**
1. Perform a query of the clinvar db with ‘ALK’and ‘EGFR’ as the search term.
2. Use efetch to get the first 100 datapoints and format it in the docsum for file output.
3. Extract the xml file based on the DocumentSummary pattern. Use the "|" as the column separator and parse the elements Id accession title chr trait_name gene_sort band clinical_significance to a final text file.

In [ ]:
!esearch -db clinvar -query "ALK OR EGFR"|\
efetch -format docsum -start 0 -stop 100|\
xtract -pattern DocumentSummary -sep "|" -element Id accession title chr trait_name gene_sort band clinical_significance > output_3.txt

#### **Question 4**
Find the mutation/s that determines whether ‘ALK’ and ‘EGFR’ will be toxic or non-toxic for non-small cell squamous lung cancer(NSCLC) patient drug treatment. Create a table that has the headers Toxic and Non-toxic. The lines will have the gene and the mutation. Example, ALK - 92C>G (p.Pro31Arg). The dash is the tab separator

*Thought Process*
1. I use esearch to extract the refseqID from the clinvar database. My query will be for ALK or EGFR and NSCLC.
2. From this I want to make sure that I only use results that have lung carcinoma (per my office hours with the Professor). I can achieve this by using grep.
3. Next I can use a python script to get only the unique refseq- not including the variation. Please refer to my python code comments for a breakdown of each line.
4. I will use these unique ID's as my search in COSMIC. I look under the drug resistance histogram and populate my table manually (like we discussed in office hours)
5. Drug Resistant Mutations go under toxic,
6. I cannot manually write down all mutations (there an few hundred for each refseqID) so I have chosen a few to use as my example
   **Note: There were mutations that had N/A for drug resistance, so I included this as an extra column in my excel sheet**

In [ ]:
!esearch -db clinvar -query "(ALK OR EGFR) AND NSCLC"|\
efetch -format docsum |\
xtract -pattern DocumentSummary -sep "|" -element title trait_name |\
grep -i 'lung carcinoma' > refseqID_output4.txt

#get unique NM- & parse based on lung cancer
#then put into cosmic- resistant mutation yes = toxic
#write up and explain what I did
#want to only use the first part NM_005228- https://cancer.sanger.ac.uk/cosmic/csamples/details?all_data=&coords=AA%3AAA&dr=&drug_pie=187320824&end=1211&gd=&id=251203&ln=EGFR&seqlen=1211&start=1#
#go into drug resistance, look up resistant mutation

In [ ]:
import re

with open("refseqID_output4.txt", 'r') as file: #open my file for reading
    lines = file.readlines()

uniqID = {} #initialize my dictionary of unique ID's
refSeqPattern = re.compile(r"(NM_\d+)(?=\.)") #define the pattern for the ref seq, I want everything after the first NM_ and before the first .
mutationPattern = re.compile(r"(?<=:)[^\s]+") #define the pattern for the mutatino, I want everything between : and the first space it encounters

for line in lines:
    refseq_match = refSeqPattern.search(line)
    mutation_match = mutationPattern.search(line)
    # If a match is found, add it to the dictionary
    
    if match:
        # Get the matches
        nm_id = refseq_match.group(1)
        mutation= mutation_match.group(0)
        
        # only add it to the dictionary if it is not present!
        if nm_id not in uniqID:
            uniqID[nm_id] = [mutation]  # Initialize as a list with the first mutation
        else:
            uniqID[nm_id].append(mutation)  # Append subsequent mutations to the list
# Print the resulting dictionary 
display(uniqID)
    


The unique refseqID's are: NM_005228, NM_004304, NM_004985
I will manually put this into the COSMIC database and look up some of the mutations independently. Based on what the drug resistance information is for each mutation, I will update my sheet accordingly.

NM_004985 is KRAS so I will *NOT* be including this in my final sheet. 

#### **Question 5**
1. **What are the strengths of Entrez EDirect?**

    Entrez EDirect connects the user with a suite of command line tools provided by the NCBI. This allows users to directly query, retrieve and process data from the NCBI databases. Entrez EDirect supports many databases- such a PubMed, Gene, ClinVar and SNP, which makes it a highly versatile tool in bioinformatics research. EDirect can also handle large amounts of data with its commands like esearch and efetch. By combinging this with other UNIX commands- such as grep and awk- the user is also able to manipulate the data and pipe commands and scripts for seamless data processing.
   
2. **How is it useful in analyzing a personal treatment for a given cancer?**

    Following what I just mentioned above, we can use the tools of Entrez to search for relevant biomedicat data- such as gene mutations, drug efficacy, clinical outcomes from databases such as PubMed and ClinVar. For example, the user can search for specific gene variants and link it to treatment outcomes, or drug resistance, in a matter of minutes. The tool allows the user to search through large databases for specific information that can personalize a treatment plan.

3. **What did you learn from the literature searches concerning the mutation for non-small cell squamous lung cancer(NSCLC)?**

    In my literature searches containing mutations with NSCLC I was able to find that specific mutations in these genes can determine whether a patient will respond positively or experience toxic effects from certain treatments. Studies showed that certain mutations will respond to specific drug therapies, while others will not. This highlights the importance of a personalized treatment approach. Being able to identify specific gene mutations can help optimize a treatment plan for someone with NSCLC.

4. **Which NCBI databases provided the information concerning the mutation and it’s sequence info? Describe the databases and the data that they hold.**
    
    ClinVar- A publicly accessible database that compiles information about genetic variations and their clinical significance.  ClinVar contains records of mutations, including pathogenicity, clinical significance, supporting evidence, and links to published research. It is often used for understanding the clinical relevance of genetic variants in diseases..

   Gene- A comprehensive resource that consolidates information on gene-specific data, such as gene structure, function, and variations.It includes information on gene sequences, gene structure, related protein products, gene pathways, and mutations in the gene. Gene records link to multiple external databases, such as RefSeq and OMIM, for further sequence and mutation data

   dbSNP- This database houses data on genetic variations, including single nucleotide polymorphisms (SNPs). It provides information on the location of SNPs, allele frequencies, genotype frequencies, and the phenotypic effects of the variants when available..

6. **How does the mutation inform the patients response to treatment?**

   
    In answering the questions for the final, I saw that mutations in genes like ALK and EGFR directly influence a patient's response to treatment in NSCLC by altering the behavior of cancer cells. Specific mutations can make cancer cells more susceptible to targeted therapies, for example, such as tyrosine kinase inhibitors (TKIs), which are designed to block the abnormal signaling caused by these mutations (Learned this while reading through pubmed abstracts from an esearch query!). Some mutations can enhance a drug's efficacy and, some mutations may confer resistance to these therapies and can cause toxic side effects. Therefore, identifying these mutations helps tailor treatment to maximize therapeutic benefit while minimizing toxicity and resistance.